In [87]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [88]:
raw = pd.read_excel('1-s2.0-S2352340918305092-mmc2.xlsx', sheet_name='Dataset with Generated Features')
df = raw.copy()
df.drop(['index', 'Material Composition', 'Formation_energy'], axis=1, inplace=True)
df.head()

,goldschmidt_TF,goldschmidt_TF_ionic,octahedral_factor,octahedral_factor_ionic,A_O,B_O,A_B,num_of_atoms_host_Asite0,shannon_radii_host_Asite0,host_Asite0_Ionic Radius (angstroms),...,Bsite_NdUnfilled_range,Bsite_NdValence_range,Bsite_NfUnfilled_range,Bsite_NfValence_range,Bsite_NpUnfilled_range,Bsite_NpValence_range,Bsite_NsUnfilled_range,Bsite_NsValence_range,Bsite_NUnfilled_range,EnergyAboveHull
0,1.021823,0.976828,0.414286,0.385714,2.86125,1.980,2.04125,7,1.440,1.26,...,0,0,0,0,0,0,0,0,0,29.747707
1,0.987385,0.889057,0.378571,0.464286,2.69500,1.930,1.82500,4,1.200,1.13,...,0,0,0,0,0,0,0,0,0,106.702335
2,0.976009,0.908360,0.452857,0.392857,2.80750,2.034,2.04150,6,1.340,1.00,...,0,0,0,0,0,0,0,0,0,171.608093
3,1.026809,0.865275,0.342857,0.492857,2.73000,1.880,1.81000,4,1.200,1.13,...,0,0,0,0,0,0,0,0,0,284.898190
4,0.909001,0.916519,0.452857,0.392857,2.61475,2.034,1.84875,6,1.083,1.03,...,0,0,0,0,0,0,0,0,0,270.007913


In [89]:
from sklearn.feature_selection import VarianceThreshold

# remove features with 0 variance
vt = VarianceThreshold()
vt.fit(df)
df = df.loc[:, vt.variances_ > 0 ]
df.shape

(1929, 792)

In [90]:
init_x = df.loc[:, df.columns !='EnergyAboveHull']
init_y = df.loc[:, 'EnergyAboveHull']

# remove low correlated features
low_corr = []
for col in init_x.columns:
    if abs(init_x[col].corr(init_y)) < 1e-2:
        low_corr.append(col)
        
from collections import defaultdict
corr_df = init_x.corr()
# get features that have correlation > 0.95 
def high_corr(corr):
    raw_corrs = defaultdict(float)
    for col in corr.columns:
        index = corr[col].index
        for pos in range(len(index)):
            if np.abs(corr[col][pos]) >= 0.90 and index[pos] != col:
                raw_corrs[(col, index[pos])] =  corr[col][pos]
    
    # remove duplicates by score
    result = defaultdict(float)
    for key,value in raw_corrs.items():
        if value not in result.values():
            result[key] = value
            
    return result

high_corrs = high_corr(corr_df)

# keep only one of the features that are highly correlated
# keep the one that has highest correlation with target
df.drop(low_corr, axis=1, inplace=True)

for key, val in high_corrs.items():
    try: 
        if np.abs(df[key[0]].corr(df['EnergyAboveHull'])) > np.abs(df[key[1]].corr(df['EnergyAboveHull'])):
            df = df.loc[:, df.columns != key[1]]
        else: 
            df = df.loc[:, df.columns != key[0]]
    except KeyError:
        continue
        
print(df.shape)

(1929, 397)


In [16]:
class_df = df.copy()
class_df['class'] = class_df['EnergyAboveHull']
class_df['class'][class_df['EnergyAboveHull'] > 40] = 1
class_df['class'][class_df['EnergyAboveHull'] <= 40] = 0
class_df.drop(['EnergyAboveHull'], axis=1, inplace=True)

/Users/williamyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/williamyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [17]:
X_class = class_df.loc[:, class_df.columns !='class']
y_class = class_df.loc[:, 'class']

In [18]:
from sklearn.feature_selection import f_regression, f_classif, SelectKBest, RFECV, RFE
from sklearn.model_selection import train_test_split

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class)

f_values, p_values = f_classif(X_train_class, y_train_class)

pval_df = pd.DataFrame({'features': X_class.columns, 'p_val': p_values})
pval_df = pval_df[pval_df['p_val'] >= 0.05]

class_df.drop(pval_df['features'], axis=1, inplace=True)
X_class.drop(pval_df['features'], axis=1, inplace=True)

/Users/williamyu/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [326] are constant.
  UserWarning)
/Users/williamyu/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/williamyu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class)

In [20]:
rfe_gb = RFECV(GradientBoostingClassifier(random_state=0), scoring='f1')
rfe_gb.fit(X_train_class, y_train_class)
print('Number of Features:',  rfe_gb.n_features_)

Number of Features: 291


In [21]:
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, ExtraTreesRegressor
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV, KFold, StratifiedKFold,RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
rfe_et_class = RFECV(ExtraTreesClassifier(random_state=0), cv=StratifiedKFold(5, shuffle=True, random_state=0), scoring='f1')
rfe_et_class.fit(X_train_class, y_train_class)
print('Number of Features:',  rfe_et_class.n_features_)

Number of Features: 140


In [22]:
rfe_tree = RFECV(DecisionTreeClassifier(random_state=0), cv=StratifiedKFold(5, shuffle=True, random_state=0), scoring='f1')
rfe_tree.fit(X_train_class, y_train_class)
print('Number of Features:',  rfe_tree.n_features_)

Number of Features: 215


In [23]:
X_tree = X_class.iloc[:, rfe_tree.support_]

In [24]:
import xgboost
xgb = xgboost.XGBClassifier(random_state=0, n_estimators=400, max_depth=5)

In [25]:
import smogn

def oversample(sampling_strategy, X, y):
    xtrains = []
    ytrains = []
    xtests = []
    ytests = []
    f1scores=[]
    acc = []
    if sampling_strategy == 'smote':
        sampler = SMOTE()
    elif sampling_strategy == 'over':
        sampler = RandomOverSampler()
        
    kf = StratifiedKFold(n_splits=5)
    for train_index, test_index in kf.split(X, y):
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]
        X_train_oversampled, y_train_oversampled = sampler.fit_sample(X_train, y_train)
        xgb.fit(X_train_oversampled, y_train_oversampled)
        y_pred = xgb.predict(X_test)
        acc.append(xgb.score(X_test, y_test))
        f1scores.append(f1_score(y_test, y_pred))

    return acc, f1scores

#xtrains, ytrains, xtests, ytests = oversample('over', X_class.to_numpy(), y_class)
acc, f1scores = oversample('over', X_tree.to_numpy(), y_class)
print('Acc: ' + str(np.mean(acc)))
print('f1: ' + str(np.mean(f1scores)))

Acc: 0.7569194536033914
f1: 0.8176525770294752


In [26]:
# oversampled train dataset for the first fold is xtrains[0]
# ytrains[0] are the corresponding labels for xtrains[0]
firstfoldtrain = pd.DataFrame(xtrains[0])
firstfoldtrain['class'] = ytrains[0]
firstfoldtrain.columns = class_df.columns
firstfoldtrain

NameError: name 'xtrains' is not defined

## Reg

In [91]:
X_reg = df.loc[:, df.columns !='EnergyAboveHull']
y_reg = df.loc[:, 'EnergyAboveHull']

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg)

# remove statistically insignificant features based on lin reg
f_values, p_values = f_regression(X_train_reg, y_train_reg)
pval_df = pd.DataFrame({'features': X_reg.columns, 'p_val': p_values})
pval_df = pval_df[pval_df['p_val'] >= 0.05]

X_reg.drop(pval_df['features'], axis=1, inplace=True)
X_reg.shape

/Users/williamyu/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/williamyu/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/williamyu/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/williamyu/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/Users/williamyu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

(1929, 338)

In [68]:
# apply inverse hyperbolic sine transform to normalize data
# logarithmic apply to test data as well because model is fitted to log data
y_reg_log = y_reg.apply(lambda x: np.arcsinh(x))

In [69]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg_log)

In [70]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

rfe_log = RFECV(DecisionTreeRegressor(random_state=0), cv=KFold(10, shuffle=True), scoring='r2')
rfe_log.fit(X_train_reg, y_train_reg)
print('Number of Features:',  rfe_log.n_features_)

Number of Features: 17


In [71]:
X_dtr = X_reg.iloc[:, rfe_log.support_]
X_dtr_train, X_dtr_test, y_dtr_train, y_dtr_test = train_test_split(X_dtr, y_reg_log)

In [72]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, ExtraTreesRegressor
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

grid = RandomizedSearchCV(ExtraTreesRegressor(random_state=0), param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
grid.fit(X_dtr_train, y_dtr_train)
grid.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  5.4min finished


{'n_estimators': 1800,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 20,
 'bootstrap': False}

In [73]:
tree = ExtraTreesRegressor(random_state=0, max_depth=20, n_estimators=1800, min_samples_split=2, min_samples_leaf=1,
                          max_features='auto', bootstrap=False)

In [95]:
# oversample for regression
import smogn

def oversample_reg(df, model):
    dftrains=[]
    dftests=[]
    r2 = []
    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    for train_index, test_index in kf.split(df):
        df_train = df.iloc[train_index]
        df_test = df.iloc[test_index]
        df_smogn = smogn.smoter(data = df_train.reset_index(drop=True), y = 'EnergyAboveHull')
        X_train_smogn = df_smogn.loc[:, df_smogn.columns!='EnergyAboveHull']
        y_train_smogn = df_smogn['EnergyAboveHull']
        X_test = df_test.loc[:, df_test.columns!='EnergyAboveHull']
        y_test = df_test['EnergyAboveHull']
        model.fit(X_train_smogn, y_train_smogn)
        r2.append(model.score(X_test, y_test))
        #dftrains.append(df_smogn)
        #dftests.append(df_test)
        
    #return dftrains, dftests
    return r2

r2 = oversample_reg(df, tree)
np.mean(r2)

r_index: 100%|##########| 168/168 [00:03<00:00, 46.65it/s]


0.7276484481812595